In [3]:
#https://dpird-dma.github.io/blog/Raster-extent-to-polygon/

'/home/jovyan/isce_sat2/contributors/naheem/notebook'

In [8]:
input_dir = '/home/jovyan/isce_sat2/contributors/naheem/download_files/coherence'
file_types = ('.tiff')
export_dir = os.path.dirname(input_dir)
export_file_name = os.path.basename(input_dir)

In [9]:
raster_list = []

for root, dirs, files in os.walk(input_dir):
    for file in files:
        if file.endswith(file_types):
            raster_list.append(os.path.join(root, file))

len(raster_list)

13

In [10]:
def index(raster):
    try:
        dataset = rasterio.open(raster)
        proj = dataset.crs.to_string()
        bounds = dataset.bounds
        geom = box(*bounds)
        raster_name = os.path.basename(raster)
        return({'name':raster_name,'path':raster,'crs':proj,'geometry':geom})
    except:
        print('could not open',raster)

In [11]:
with Pool() as p:
    polygons = list(tqdm(p.imap(index, raster_list), total=len(raster_list)))

100%|██████████| 13/13 [00:00<00:00, 607.43it/s]


In [12]:
all_polygons = pd.DataFrame(polygons)
crs_list = all_polygons['crs'].unique()
print(crs_list)
all_polygons.head()

['EPSG:4326']


,name,path,crs,geometry
0,silver_34715_20011-001_20016-002_0019d_s01_L09...,/home/jovyan/isce_sat2/contributors/naheem/dow...,EPSG:4326,"POLYGON ((-116.55637932 42.94838004, -116.5563..."
1,irnton_01406_21020-028_21022-002_0006d_s01_L09...,/home/jovyan/isce_sat2/contributors/naheem/dow...,EPSG:4326,"POLYGON ((-107.55532596 37.658568, -107.555325..."
2,uticam_21003_21004-002_21013-003_0034d_s01_L09...,/home/jovyan/isce_sat2/contributors/naheem/dow...,EPSG:4326,"POLYGON ((-109.694886 46.408712400000006, -109..."
3,dorado_34017_20014-002_20016-000_0014d_s01_L09...,/home/jovyan/isce_sat2/contributors/naheem/dow...,EPSG:4326,"POLYGON ((-119.95015080000002 38.62747884, -11..."
4,alamos_35915_20005-003_20008-000_0007d_s01_L09...,/home/jovyan/isce_sat2/contributors/naheem/dow...,EPSG:4326,"POLYGON ((-106.30489260000002 35.6886882, -106..."


In [13]:
for crs in tqdm(crs_list):
    one_crs_df = all_polygons[all_polygons['crs']==crs]
    one_crs_gdf = gpd.GeoDataFrame(one_crs_df,crs=crs) #crs={'init' :crs}
    save_name = os.path.join(export_dir,(export_file_name+" crs "+crs.split(':')[1]+'.gpkg'))
    one_crs_gdf.to_file(save_name, driver ="GPKG")
    print(save_name)

100%|██████████| 1/1 [00:00<00:00,  1.13it/s]

/home/jovyan/isce_sat2/contributors/naheem/download_files/coherence crs 4326.gpkg
